In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
texts = pd.read_csv('texts.csv')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_tfidf = TfidfVectorizer(stop_words='english').fit_transform(texts.TEXT).toarray()
text_tfidf = pd.DataFrame(text_tfidf)
text_tfidf['ID'] = texts['ID']

In [4]:
X_train = pd.merge(train, text_tfidf, on='ID', how='left').drop(['ID', 'CATEGORY'], 1).values
X_test = pd.merge(test, text_tfidf, on='ID', how='left').drop(['ID'], 1).values
y_train = pd.factorize(train['CATEGORY'])[0]

In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

nb_clf = GaussianNB()
np.mean(cross_val_score(nb_clf, X_train, y_train, cv=10, scoring='roc_auc', n_jobs=-1))

0.8717888271234406

In [6]:
nb_clf.fit(X_train, y_train)

GaussianNB(priors=None)

In [7]:
test['SPAM_PROB'] = nb_clf.predict_proba(X_test)[:, 1]

In [8]:
test.to_csv('baseline.csv', index=None)